In [1]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
import joblib
import json
import mlflow
import os

In [2]:
# Load data
df = pd.read_csv("../../p1_data/cleaned/amazon_clean.csv")
labels = df['label'].values

# Load embeddings

In [3]:
# Load embeddings
embeddings_bert = np.load("../../p2_embeddings/embeddings_bert-base-uncased.npy")
embeddings_distilbert = np.load("../../p2_embeddings/embeddings_distilbert-base-uncased.npy")
embeddings_xlmr = np.load("../../p2_embeddings/embeddings_xlm-roberta-base.npy")

In [4]:
# Start MLflow tracking

# Set explicit tracking URI to save in project root
project_root = os.path.abspath("../..")  # Goes up 2 levels from notebooks/
mlflow.set_tracking_uri(f"file:{project_root}/mlruns")
print(f"MLflow saving to: {mlflow.get_tracking_uri()}")

# Create mlruns directory if it doesn't exist
os.makedirs(f"{project_root}/mlruns", exist_ok=True)

mlflow.set_experiment("sentiment_analysis_transformer_comparison")

def log_experiment(embedding_name, classifier_name, results, classifier):
    run_name = f"{embedding_name}_{classifier_name}"
    
    # FIXED LINE: Add run_name parameter here ↓
    with mlflow.start_run(run_name=run_name):  # ← ADD run_name=run_name INSIDE THE PARENTHESES
        mlflow.log_param("embedding_model", embedding_name)
        mlflow.log_param("classifier", classifier_name)
        mlflow.log_metric("accuracy", results['accuracy'])
        mlflow.log_metric("f1_score", results['f1_score'])
        mlflow.log_metric("precision", results['precision']) 
        mlflow.log_metric("recall", results['recall'])
        mlflow.sklearn.log_model(classifier, f"{classifier_name}_model")
        mlflow.set_tag("run_name", run_name)

/opt/anaconda3/envs/env/lib/python3.13/site-packages/mlflow/tracking/_tracking_service/utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
2025/12/06 21:17:21 INFO mlflow.tracking.fluent: Experiment with name 'sentiment_analysis_transformer_comparison' does not exist. Creating a new experiment.


MLflow saving to: file:/Users/kamal/Documents/Study/DEPI/Learnings/NLP/Portfolio/mega-sentiment-project/mlruns


# Prepare Models Training PipeLine

In [5]:
time_tracking_per_model = {}

def train_classifiers(embeddings, labels, model_name):
    """Train classifiers and return both results AND trained models"""
    X_train, X_test, y_train, y_test = train_test_split(
        embeddings, labels, test_size=0.2, random_state=42, stratify=labels
    )

    # ==========. Check class imbalance ==========
    n_positive = sum(y_train)
    n_negative = len(y_train) - n_positive
    print(f"Training data: {n_positive} positive, {n_negative} negative")
    print(f"Class ratio: {n_positive/n_negative:.2f}:1 (pos:neg)")
    
    # ========== Classifiers with imbalance handling ==========
    classifiers = {
        'XGBoost': XGBClassifier(
            random_state=42,
            scale_pos_weight=n_negative/n_positive  # Penalize minority class errors more
        ),
        'LightGBM': LGBMClassifier(
            random_state=42,
            class_weight='balanced'  # Automatically adjust for imbalance
        ),
        'CatBoost': CatBoostClassifier(
            random_state=42, 
            verbose=False,
            auto_class_weights='Balanced'  # Balance class weights
        )
    }

    results = {}
    timer_dict = {}
    trained_models = {}

    for classif_name, classif in classifiers.items():
        print(f"Training {classif_name} on {model_name} embeddings...")
        
        # Train
        start_time = time.time()
        classif.fit(X_train, y_train)
        end_time = time.time()
        
        elapsed_time = end_time - start_time
        timer_dict[classif_name] = elapsed_time
        
        # Store the trained model
        trained_models[classif_name] = classif
        
        # Predict and evaluate
        y_pred = classif.predict(X_test)
        results[classif_name] = {
            'accuracy': accuracy_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred, average='macro'),
            'precision': precision_score(y_test, y_pred, average='macro'),
            'recall': recall_score(y_test, y_pred, average='macro')
        }

        # Log to MLflow
        log_experiment(model_name, classif_name, results[classif_name], classif)
        print(f"Logged {classif_name} on {model_name} to MLflow")
    
    time_tracking_per_model[model_name] = timer_dict
    
    return results, trained_models  # Return BOTH

# Run All Experiments (3 embedding × 3 models)

In [6]:
all_results = {}      # For metrics
all_models = {}       # For actual trained models

# Train on BERT embeddings
print("Start Training on BERT embeddings...")
bert_results, bert_models = train_classifiers(embeddings_bert, labels, "BERT")
all_results['bert'] = bert_results
all_models['bert'] = bert_models
print("End Training on BERT embeddings" + "_"*64)

# Train on DistilBERT embeddings  
print("Start Training on DistilBERT embeddings...")
distilbert_results, distilbert_models = train_classifiers(embeddings_distilbert, labels, "DistilBERT")
all_results['distilbert'] = distilbert_results
all_models['distilbert'] = distilbert_models
print("End Training on DistilBERT embeddings" + "_"*64)

# Train on XLM-R embeddings
print("Start Training on XLM-R embeddings...")
xlmr_results, xlmr_models = train_classifiers(embeddings_xlmr, labels, "XLM-R")
all_results['xlmr'] = xlmr_results
all_models['xlmr'] = xlmr_models
print("End Training on XLM-R embeddings" + "_"*64)

Start Training on BERT embeddings...
Training data: 245026 positive, 45538 negative
Class ratio: 5.38:1 (pos:neg)
Training XGBoost on BERT embeddings...


2025/12/06 21:17:36 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Cmd('git') failed due to: exit code(1)
  cmdline: git version
  stderr: 'xcode-select: note: No developer tools were found, requesting install.
If developer tools are located at a non-default location on disk, use `sudo xcode-select --switch path/to/Xcode.app` to specify the Xcode that you wish to use for command line developer tools, and cancel the installation dialog.
See `man xcode-select` for more details.'
2025/12/06 21:17:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/06 21:17:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logged XGBoost on BERT to MLflow
Training LightGBM on BERT embeddings...
[LightGBM] [Info] Number of positive: 245026, number of negative: 45538
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.221328 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195840
[LightGBM] [Info] Number of data points in the train set: 290564, number of used features: 768
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


/opt/anaconda3/envs/env/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
2025/12/06 21:17:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/06 21:17:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logged LightGBM on BERT to MLflow
Training CatBoost on BERT embeddings...


...

Logged XGBoost on XLM-R to MLflow
Training LightGBM on XLM-R embeddings...
[LightGBM] [Info] Number of positive: 245026, number of negative: 45538
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.240351 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195840
[LightGBM] [Info] Number of data points in the train set: 290564, number of used features: 768
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


/opt/anaconda3/envs/env/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
2025/12/06 21:21:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/06 21:21:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logged LightGBM on XLM-R to MLflow
Training CatBoost on XLM-R embeddings...


2025/12/06 21:22:50 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/06 21:22:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logged CatBoost on XLM-R to MLflow
End Training on XLM-R embeddings________________________________________________________________


In [7]:
# ========== Save models IMMEDIATELY ==========
print("\n💾 Saving trained models...")

# Save ALL models for each embedding
for embedding_name, models_dict in all_models.items():
    filename = f"../../artifacts/models/{embedding_name}_all_models.joblib"
    joblib.dump(models_dict, filename)
    print(f"Saved {len(models_dict)} models to {filename}")


💾 Saving trained models...
Saved 3 models to ../../artifacts/models/bert_all_models.joblib
Saved 3 models to ../../artifacts/models/distilbert_all_models.joblib
Saved 3 models to ../../artifacts/models/xlmr_all_models.joblib


In [8]:
# ========== Display Results ==========
for model, times in time_tracking_per_model.items():
    print(f"\nTraining times for {model}:")
    for clf, time_val in times.items():
        print(f"  {clf}: {time_val:.2f} seconds")

for model, classifiers in all_results.items():
    print(f"\nMetrics for {model}:")
    for clf, metrics in classifiers.items():
        print(f"  {clf}: Accuracy={metrics['accuracy']:.4f}, F1={metrics['f1_score']:.4f}")


Training times for BERT:
  XGBoost: 14.71 seconds
  LightGBM: 16.19 seconds
  CatBoost: 72.65 seconds

Training times for DistilBERT:
  XGBoost: 15.27 seconds
  LightGBM: 16.66 seconds
  CatBoost: 72.07 seconds

Training times for XLM-R:
  XGBoost: 14.88 seconds
  LightGBM: 16.97 seconds
  CatBoost: 75.29 seconds

Metrics for bert:
  XGBoost: Accuracy=0.8392, F1=0.7495
  LightGBM: Accuracy=0.8144, F1=0.7283
  CatBoost: Accuracy=0.8480, F1=0.7622

Metrics for distilbert:
  XGBoost: Accuracy=0.8468, F1=0.7594
  LightGBM: Accuracy=0.8204, F1=0.7354
  CatBoost: Accuracy=0.8552, F1=0.7724

Metrics for xlmr:
  XGBoost: Accuracy=0.8460, F1=0.7581
  LightGBM: Accuracy=0.8226, F1=0.7383
  CatBoost: Accuracy=0.8562, F1=0.7739


In [9]:
# ========== Find Best Model ==========
def find_best_model(all_results):
    best_accuracy = 0
    best_combo = {}
    
    for embedding_name, classifiers in all_results.items():
        for classifier_name, metrics in classifiers.items():
            if metrics['accuracy'] > best_accuracy:
                best_accuracy = metrics['accuracy']
                best_combo = {
                    'embedding': embedding_name,
                    'classifier': classifier_name,
                    'metrics': metrics,
                    'embedding_path': f"../../p2_embeddings/embeddings_{'bert-base-uncased' if embedding_name == 'bert' else 'distilbert-base-uncased' if embedding_name == 'distilbert' else 'xlm-roberta-base'}.npy"
                }
    
    return best_combo

# Find best model
best = find_best_model(all_results)
print(f"\nBest Model: {best['classifier']} on {best['embedding']} embeddings")
print(f"Accuracy: {best['metrics']['accuracy']:.4f}")


Best Model: CatBoost on xlmr embeddings
Accuracy: 0.8562


In [10]:
# ========== Save Best Model Info ==========
best_model_info = {
    'embedding_model': best['embedding'],
    'classifier': best['classifier'],
    'metrics': best['metrics'],
    'model_file': f"{best['embedding']}_{best['classifier']}.joblib"  # NEW: Which file to load
}

with open('../../artifacts/models/best_model_info.json', 'w') as f:
    json.dump(best_model_info, f, indent=4)

print("Saved best model info to artifacts/models/best_model_info.json")

Saved best model info to artifacts/models/best_model_info.json


In [11]:
# ========== Save Best Model Separately (for easy API loading) ==========
print(f"\nSaving best model separately for API...")

# Get the actual best model from our stored models
best_model_object = all_models[best['embedding']][best['classifier']]

# Save it
best_model_path = f"../../artifacts/models/{best['embedding']}_{best['classifier']}.joblib"
joblib.dump(best_model_object, best_model_path)
print(f"Saved best model to {best_model_path}")

# ========== Test the saved model ==========
print(f"\nTesting saved model...")
test_embedding = np.random.randn(1, 768)  # Random test data
loaded_model = joblib.load(best_model_path)
prediction = loaded_model.predict(test_embedding)
print(f"Model test passed! Prediction shape: {prediction.shape}")

print("\n" + "="*60)
print("ALL DONE! Models are saved and ready for API!")
print("="*60)


Saving best model separately for API...
Saved best model to ../../artifacts/models/xlmr_CatBoost.joblib

Testing saved model...
Model test passed! Prediction shape: (1,)

ALL DONE! Models are saved and ready for API!
